# CAP4453 Robot Vision: Softmax Classifier

This notebook provides a skeleton implementation of a one‑layer linear classifier using PyTorch. You will load the CIFAR‑10 or MNIST dataset, flatten the images, and train a linear classifier with softmax loss. The code below is only a starting point; feel free to modify it as needed. The model will automatically use a GPU if available.


In [1]:
import torch
# import torch_directml
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Choose device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch_directml.device()
print(f"Using device: {device}")

# Select dataset: 'cifar10' or 'mnist'
dataset_name = 'cifar10'

if dataset_name.lower() == 'cifar10':
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    num_classes = 10
    input_dim = 32*32*3
elif dataset_name.lower() == 'mnist':
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    num_classes = 10
    input_dim = 28*28
else:
    raise ValueError('Unknown dataset')

# Split train into train/val
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")


Using device: cpu
Files already downloaded and verified


c:\Users\Kauan Lima\Documents\!UCF\UCF-Assignments\CAP4453\.venv311\Lib\site-packages\torchvision\datasets\cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


Files already downloaded and verified
Train samples: 45000, Validation samples: 5000, Test samples: 10000


In [ ]:
# Define a simple linear classifier using PyTorch
import torch.nn as nn

class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)  # Score function: f(x) = Wx + b

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        return self.fc(x)  # Compute class scores via matrix multiplication

# Hyperparameter combinations to test
learning_rates = [1e-4, 1e-3, 1e-2, 1e-1]
weight_decays = [0, 1e-5, 1e-4, 1e-3]
batch_sizes = [64, 128, 256]
num_epochs = 15

# Store results for all experiments
all_results = []
best_val_acc = 0
best_config = {}

print("=" * 80)
print("HYPERPARAMETER SEARCH FOR LINEAR CLASSIFIER")
print("=" * 80)

# Test all combinations
for batch_size in batch_sizes:
    # Create data loaders with current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    for lr in learning_rates:
        for wd in weight_decays:
            print(f"\n{'='*80}")
            print(f"Testing: batch_size={batch_size}, lr={lr}, weight_decay={wd}")
            print(f"{'='*80}")
            
            # Initialize model, loss function (softmax) and SGD optimizer
            model = LinearClassifier(input_dim, num_classes).to(device)
            criterion = nn.CrossEntropyLoss()  # Softmax loss
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
            
            # Track metrics for this configuration
            train_losses = []
            train_accs = []
            val_accs = []
            
            # Training loop with Stochastic Gradient Descent (SGD)
            for epoch in range(num_epochs):
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                
                # Sample minibatches and train
                for images, labels in train_loader:
                    images, labels = images.to(device), labels.to(device)
                    
                    # Compute scores
                    optimizer.zero_grad()
                    outputs = model(images)  # Score function
                    
                    # Compute softmax loss
                    loss = criterion(outputs, labels)
                    
                    # Compute gradients
                    loss.backward()
                    
                    # Update parameters using SGD
                    optimizer.step()
                    
                    running_loss += loss.item() * images.size(0)
                    _, predicted = outputs.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()
                
                train_loss = running_loss / total
                train_acc = 100. * correct / total
                
                # Monitor validation accuracy
                model.eval()
                val_correct = 0
                val_total = 0
                with torch.no_grad():
                    for images, labels in val_loader:
                        images, labels = images.to(device), labels.to(device)
                        outputs = model(images)
                        _, predicted = outputs.max(1)
                        val_total += labels.size(0)
                        val_correct += predicted.eq(labels).sum().item()
                val_acc = 100. * val_correct / val_total
                
                train_losses.append(train_loss)
                train_accs.append(train_acc)
                val_accs.append(val_acc)
                
                print(f"Epoch {epoch+1}/{num_epochs}, Train loss: {train_loss:.4f}, Train acc: {train_acc:.2f}%, Val acc: {val_acc:.2f}%")
            
            final_val_acc = val_accs[-1]
            
            # Store results
            all_results.append({
                'batch_size': batch_size,
                'learning_rate': lr,
                'weight_decay': wd,
                'final_val_acc': final_val_acc,
                'train_losses': train_losses,
                'train_accs': train_accs,
                'val_accs': val_accs
            })
            
            # Track best configuration
            if final_val_acc > best_val_acc:
                best_val_acc = final_val_acc
                best_config = {
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'weight_decay': wd,
                    'val_acc': final_val_acc,
                    'train_losses': train_losses,
                    'train_accs': train_accs,
                    'val_accs': val_accs
                }
            
            print(f"\nFinal validation accuracy: {final_val_acc:.2f}%")

print("\n" + "=" * 80)
print("BEST CONFIGURATION FOUND:")
print("=" * 80)
print(f"Batch Size: {best_config['batch_size']}")
print(f"Learning Rate: {best_config['learning_rate']}")
print(f"Weight Decay: {best_config['weight_decay']}")
print(f"Best Validation Accuracy: {best_config['val_acc']:.2f}%")
print("=" * 80)

Epoch 1/10, Train loss: 2.0603, Train acc: 27.48%, Val acc: 31.66%
Epoch 2/10, Train loss: 1.9212, Train acc: 33.57%, Val acc: 32.78%
Epoch 3/10, Train loss: 1.8754, Train acc: 35.34%, Val acc: 34.40%
Epoch 4/10, Train loss: 1.8474, Train acc: 36.18%, Val acc: 34.90%
Epoch 5/10, Train loss: 1.8300, Train acc: 36.98%, Val acc: 35.72%
Epoch 6/10, Train loss: 1.8165, Train acc: 37.52%, Val acc: 36.20%
Epoch 7/10, Train loss: 1.8061, Train acc: 37.82%, Val acc: 36.98%
Epoch 8/10, Train loss: 1.7973, Train acc: 38.23%, Val acc: 36.56%
Epoch 9/10, Train loss: 1.7901, Train acc: 38.55%, Val acc: 36.60%
Epoch 10/10, Train loss: 1.7835, Train acc: 38.75%, Val acc: 36.64%


### Next steps

- Experiment with different learning rates, regularization strengths and batch sizes. Please record (e.g., screenshot, copy training/val log history etc) the validation set performances with different hyper‑parameters in the report. 
- After choosing hyper‑parameters, train on the combined train+validation set and report the test accuracy.
- Plot training and validation accuracies over epochs.
- Compare these results with those obtained using a two‑layer network (see the next notebook).


In [ ]:
# Display summary of all experiments
import pandas as pd

print("\n" + "=" * 80)
print("SUMMARY OF ALL EXPERIMENTS")
print("=" * 80)

df_results = pd.DataFrame([{
    'Batch Size': r['batch_size'],
    'Learning Rate': r['learning_rate'],
    'Weight Decay': r['weight_decay'],
    'Final Val Acc': f"{r['final_val_acc']:.2f}%"
} for r in all_results])

# Sort by validation accuracy (descending)
df_results = df_results.sort_values(by='Final Val Acc', ascending=False)
print(df_results.to_string(index=False))

In [ ]:
# Plot training curves for best configuration
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot accuracy
axes[0].plot(range(1, len(best_config['train_accs']) + 1), best_config['train_accs'], 
             'b-', label='Train Accuracy', marker='o')
axes[0].plot(range(1, len(best_config['val_accs']) + 1), best_config['val_accs'], 
             'r-', label='Validation Accuracy', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Linear Classifier: Training and Validation Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(range(1, len(best_config['train_losses']) + 1), best_config['train_losses'], 
             'g-', label='Train Loss', marker='o')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Linear Classifier: Training Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('linear_classifier_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nBest Configuration:")
print(f"Batch Size: {best_config['batch_size']}")
print(f"Learning Rate: {best_config['learning_rate']}")
print(f"Weight Decay: {best_config['weight_decay']}")
print(f"Final Validation Accuracy: {best_config['val_accs'][-1]:.2f}%")

## Final Evaluation on Test Set

Now train with best hyperparameters on combined train+validation set and evaluate on test set.

In [ ]:
# Combine train and validation sets
from torch.utils.data import ConcatDataset

combined_dataset = ConcatDataset([train_dataset, val_dataset])
combined_loader = DataLoader(combined_dataset, batch_size=best_config['batch_size'], 
                            shuffle=True, num_workers=2)

print("=" * 80)
print("FINAL TRAINING ON TRAIN+VALIDATION SET")
print("=" * 80)
print(f"Using best hyperparameters:")
print(f"  Batch Size: {best_config['batch_size']}")
print(f"  Learning Rate: {best_config['learning_rate']}")
print(f"  Weight Decay: {best_config['weight_decay']}")
print(f"\nTraining on {len(combined_dataset)} samples...")
print("=" * 80)

# Initialize final model with best hyperparameters
final_model = LinearClassifier(input_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(final_model.parameters(), 
                            lr=best_config['learning_rate'], 
                            weight_decay=best_config['weight_decay'])

# Train on combined dataset
num_epochs_final = 20
for epoch in range(num_epochs_final):
    final_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in combined_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    train_loss = running_loss / total
    train_acc = 100. * correct / total
    print(f"Epoch {epoch+1}/{num_epochs_final}, Train loss: {train_loss:.4f}, Train acc: {train_acc:.2f}%")

# Evaluate on test set
print("\n" + "=" * 80)
print("FINAL TEST SET EVALUATION")
print("=" * 80)

# Create test loader with best batch size
test_loader = DataLoader(test_dataset, batch_size=best_config['batch_size'], 
                        shuffle=False, num_workers=2)

final_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_acc = 100. * test_correct / test_total
print(f"\n✓ FINAL TEST ACCURACY: {test_acc:.2f}%")
print("=" * 80)